<h1>Neural Network Using only Maths, Numpy, and Pandas<h1>

<h2>Compile images into DataFrame<h2>

In [115]:
# requirements: pip install pillow numpy pandas
import os, glob
from PIL import Image
import numpy as np
import pandas as pd

def load_image_folder(dataset_path, img_size=(32,32), as_gray=True):
    """
    Read images from dataset/<label>/*.png
    Returns:
      X: np.array shape (n_samples, img_size[0]*img_size[1]) dtype float32 normalized 0..1
      y: np.array shape (n_samples,) integer labels
      df: pandas DataFrame with filepaths & string labels
      label_to_idx: dict mapping label string -> int
    """
    files = glob.glob(os.path.join(dataset_path, '*', '*.png'))
    files.sort()
    images, labels = [], []
    for f in files:
        folder_name = os.path.basename(os.path.dirname(f))
        lbl = folder_name.rsplit("_", 1)[-1]
        img = Image.open(f)
        if as_gray:
            img = img.convert('L')          # single channel
        img = img.resize(img_size)         # consistent size
        arr = np.array(img, dtype=np.float32) / 255.0
        images.append(arr.flatten())
        labels.append(lbl)

    X = np.stack(images, axis=0)
    unique_labels = sorted(set(labels))
    label_to_idx = {lab:i for i,lab in enumerate(unique_labels)}
    Y = np.array([label_to_idx[l] for l in labels], dtype=np.int32)

    return X, Y, label_to_idx


<h2>Load Training, Validation, and Testing data<h2>

In [116]:
# One hot encoding funtion
def one_hot_encode(y, num_classes):
    encoded = np.eye(num_classes)[y].astype(np.float32)
    return encoded

# load Train data
X, Y, label_to_idx = load_image_folder('DevanagariHandwrittenCharacterDataset/Train', img_size=(32,32))
n_classes = len(label_to_idx)
Y_ohe = one_hot_encode(Y, n_classes)

# simple train/val split
N = X.shape[0]
idx = np.random.permutation(N)
train_idx = idx[:int(0.8*N)]
val_idx   = idx[int(0.8*N):]
X_train, Y_train, Y_train_ohe = X[train_idx].T, Y[train_idx].T, Y_ohe[train_idx].T
X_val, Y_val = X[val_idx].T, Y[val_idx].T
X_mean = np.mean(X_train, axis=1, keepdims=True)
X_train -= X_mean
X_val   -= X_mean

# Print shapes of data
print("--------------------------------------------")
print("Shapes of Training and Validation Data:")
print("X_train.shape = ", X_train.shape)
print("Y_train.shape = ", Y_train.shape)
print("Y_train_ohe.shape = ", Y_train_ohe.shape)
print("X_val.shape = ", X_val.shape)
print("Y_val.shape = ", Y_val.shape)
print("--------------------------------------------")

# load Test data
X_test, Y_test, label_to_idx = load_image_folder('DevanagariHandwrittenCharacterDataset/Test', img_size=(32,32))
X_test = X_test.T
Y_test = Y_test.T

X_test  -= X_mean
# Print shapes of test data
print("Shapes of Test Data:")
print("X_test.shape = ", X_test.shape)
print("Y_test.shape = ", Y_test.shape)
print("--------------------------------------------")



--------------------------------------------
Shapes of Training and Validation Data:
X_train.shape =  (1024, 62560)
Y_train.shape =  (62560,)
Y_train_ohe.shape =  (46, 62560)
X_val.shape =  (1024, 15640)
Y_val.shape =  (15640,)
--------------------------------------------
Shapes of Test Data:
X_test.shape =  (1024, 13800)
Y_test.shape =  (13800,)
--------------------------------------------


<h2>1) Initialize Weights<h2>

In [117]:
def init_weights():
    W0 = np.random.randn(512, 1024) * np.sqrt(2.0 / 1024)       # (512,1024)
    b0 = np.zeros((512,1))            # (512,1) 
    W1 = np.random.randn(256, 512) * np.sqrt(2.0 / 512)        # (256, 512)
    b1 = np.zeros((256,1))            # (256,1)
    W2 = np.random.randn(128, 256) * np.sqrt(2.0 / 256)         # (128, 256)
    b2 = np.zeros((128,1))            # (128,1) 
    W3 = np.random.randn(46, 128) * np.sqrt(2.0 / 128)         # (46, 128)
    b3 = np.zeros((46,1))             # (46,1)
    return W0, b0, W1, b1, W2, b2, W3, b3

<h2>2) Define helper functions<h2>

In [118]:
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # for numerical stability
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def get_predictions(Y_pred_ohe):
    return np.argmax(Y_pred_ohe, axis=0)

def check_accuracy(Y_pred, Y):
    accuracy = np.sum(Y_pred == Y) / np.size(Y)
    return accuracy

def cross_entropy_loss(O2, Y_ohe):
    N = O2.shape[1]
    eps = 1e-12
    loss = -np.sum(Y_ohe * np.log(O2 + eps)) / N
    return loss

def clip_gradients(grads, max_norm=1.0):
    for g in grads:
        norm = np.linalg.norm(g)
        if norm > max_norm:
            g *= max_norm / (norm + 1e-12)
    return grads

<h2>Neural Network Process<h2>
<h3>i) Forward Propagation<h3>

In [119]:
def forward_pass(X, W0, b0, W1, b1, W2, b2, W3, b3):
    # Layer 0
    print(f"Z0 = W0 @ X + b0 => {W0.shape} @ {X.shape} + {b0.shape}")
    Z0 = W0 @ X + b0
    # print(f"Z0 = W0 @ X + b0 = {Z0}")
    O0 = relu(Z0)       # (256, 62560)
    # print(f"O0 = relu(Z0) = {O0}")
    # Layer 1
    print(f"Z1 = W1 @ O0 + b1 => {W1.shape} @ {O0.shape} + {b1.shape}")
    Z1 = W1 @ O0 + b1
    # print(f"Z1 = W1 @ O0 + b1 = {Z1}")
    O1 = relu(Z1)       # (128, 62560)
    # print(f"O1 = relu(Z1) = {O1}")
    # Layer 2 (Output layer)
    print(f"Z2 = W2 @ O1 + b2 => {W2.shape} @ {O1.shape} + {b2.shape}")
    Z2 = W2 @ O1 + b2
    # print(f"Z2 = W2 @ O1 + b2 = {Z2}")
    O2 = relu(Z2)    # (46, 62560)
    # print(f"O2 = softmax(Z2) = {O2}")
    Z3 = W3 @ O2 + b3
    O3 = softmax(Z3)
    return Z0, O0, Z1, O1, Z2, O2, Z3, O3

<h3>ii) Backward Propagation<h3>

In [120]:
def backward_pass(X, Y_ohe, Z0, O0, Z1, O1, Z2, O2, Z3, O3, W0, b0, W1, b1, W2, b2, W3, b3):
    N = X.shape[1]
    # Output Layer
    d3 = O3 - Y_ohe     # (46, 62560)

    dLdW3 = (d3 @ O2.T) / N       # (46, 128) = (46, 62560) X (62560, 128)

    dLdb3 = np.sum(d3, axis=1, keepdims=True) / N      # (46, 1) = sum over columns of (46, 62560)

    dLdO2 = W3.T @ d3       # (128, 62560) = (128, 46) X (46, 62560)

    d2 = dLdO2 * (Z2 > 0)    # (128, 62560) ReLU Backprop

    dLdW2 = (d2 @ O1.T) / N       # (46, 128) = (46, 62560) X (62560, 128)

    dLdb2 = np.sum(d2, axis=1, keepdims=True) / N      # (46, 1) = sum over columns of (46, 62560)

    # Backprop to Layer 1
    dLdO1 = W2.T @ d2       # (128, 62560) = (128, 46) X (46, 62560)

    d1 = dLdO1 * (Z1 > 0)    # (128, 62560) ReLU Backprop

    dLdW1 = (d1 @ O0.T) / N       # (128, 256) = (128, 62560) X (62560, 256)

    dLdb1 = np.sum(d1, axis=1, keepdims=True) / N      # (128, 1) = sum over columns of (128, 62560)

    # Backprop to Layer 0
    dLdO0 = W1.T @ d1       # (256, 62560) = (256, 128) X (128, 62560)

    d0 = dLdO0 * (Z0 > 0)    # (256, 62560) ReLU Backprop

    dLdW0 = (d0 @ X.T) / N        # (256, 1024) = (256, 62560) X (62560, 1024)

    dLdb0 = np.sum(d0, axis=1, keepdims=True) / N      # (256, 1) = sum over columns of (256, 62560)

    # # Print Gradients
    # print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-\nGradients:")
    # print(f"dLdW0 = {dLdW0}")
    # print(f"dLdb0 = {dLdb0}")
    # print(f"dLdW1 = {dLdW1}")
    # print(f"dLdb1 = {dLdb1}")
    # print(f"dLdW2 = {dLdW2}")
    # print(f"dLdb2 = {dLdb2}")
    # print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-\n")

    # Return gradients
    return clip_gradients([dLdW0, dLdb0, dLdW1, dLdb1, dLdW2, dLdb2, dLdW3, dLdb3], max_norm=1.0)

<h3>iii) Gradient Descent Updates<h3>

In [121]:
def update_params(W0, b0, W1, b1, W2, b2, W3, b3, dLdW0, dLdb0, dLdW1, dLdb1, dLdW2, dLdb2, dLdW3, dLdb3, lr = 0.001):
    W0 = W0 - lr * dLdW0
    b0 = b0 - lr * dLdb0
    W1 = W1 - lr * dLdW1
    b1 = b1 - lr * dLdb1
    W2 = W2 - lr * dLdW2
    b2 = b2 - lr * dLdb2
    W3 = W3 - lr * dLdW3
    b3 = b3 - lr * dLdb3

    return W0, b0, W1, b1, W2, b2, W3, b3

<h3>iv) Training Loop<h3>

In [122]:
def train(X_train, Y_train, Y_train_ohe, X_val, Y_val, epochs, lr):
    # Initialize Parameters
    W0, b0, W1, b1, W2, b2, W3, b3 = init_weights()

    # print shapes of weights and biases
    print("Shapes of Weights and Biases:")
    print("W0.shape = ", W0.shape)
    print("b0.shape = ", b0.shape)
    print("W1.shape = ", W1.shape)
    print("b1.shape = ", b1.shape)
    print("W2.shape = ", W2.shape)
    print("b2.shape = ", b2.shape)
    print("W3.shape = ", W3.shape)
    print("b3.shape = ", b3.shape)
    print("--------------------------------------------")
    print("W0 = ", W0)
    print("b0 = ", b0)
    print("W1 = ", W1)
    print("b1 = ", b1)
    print("W2 = ", W2)
    print("b2 = ", b2)
    print("W3 = ", W3)
    print("b3 = ", b3)

    print("------------------------------------\nTraining Loop:")
    for epoch in range(1, epochs+1):
        # Forward Pass and Accuracy check
        print(f"********* Epoch {epoch} *********")
        Z0, O0, Z1, O1, Z2, O2, Z3, O3 = forward_pass(X_train, W0, b0, W1, b1, W2, b2, W3, b3)    # F.P. training data
        loss = cross_entropy_loss(O3, Y_train_ohe)
        print(f"Loss = {loss:.6f}, train_acc = {check_accuracy(get_predictions(O3), Y_train)}")

        # _, _, _, _, _, O2_val = forward_pass(X_val, W0, b0, W1, b1, W2, b2)    # F.P. validation data
        # print(f"Validation Accuracy: {check_accuracy(get_predictions(O2_val), Y_val)}")

        # Back Prop
        dLdW0, dLdb0, dLdW1, dLdb1, dLdW2, dLdb2, dLdW3, dLdb3 = backward_pass(X_train, Y_train_ohe, Z0, O0, Z1, O1, Z2, O2, Z3, O3, W0, b0, W1, b1, W2, b2, W3, b3)
        
        # Update Parameters
        W0, b0, W1, b1, W2, b2, W3, b3 = update_params(W0, b0, W1, b1, W2, b2, W3, b3,
                                               dLdW0=dLdW0,
                                               dLdb0=dLdb0,
                                               dLdW1=dLdW1,
                                               dLdb1=dLdb1,
                                               dLdW2=dLdW2,
                                               dLdb2=dLdb2,
                                               dLdW3=dLdW3,
                                               dLdb3=dLdb3,
                                               lr=lr)

    print("------------------------------------")
    return [W0, b0, W1, b1, W2, b2, W3, b3]

<h3>Get Predictions<h3>

In [123]:
def predict(X, Y, W0, b0, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, _, _, O3 = forward_pass(X, W0, b0, W1, b1, W2, b2, W3, b3)
    predictions = get_predictions(O3)
    print(f"Predictions: {predictions}")
    print(f"Prediction Accuracy = {check_accuracy(predictions, Y)}")
    return predictions

<h2>Run The Process<h2>

In [124]:
parameters = train(X_train, Y_train, Y_train_ohe, X_val, Y_val, epochs=200, lr=0.1)
Y_test_predictions = predict(X_test, Y_test, *parameters)

Shapes of Weights and Biases:
W0.shape =  (512, 1024)
b0.shape =  (512, 1)
W1.shape =  (256, 512)
b1.shape =  (256, 1)
W2.shape =  (128, 256)
b2.shape =  (128, 1)
W3.shape =  (46, 128)
b3.shape =  (46, 1)
--------------------------------------------
W0 =  [[-0.00691867  0.05790788  0.00393727 ... -0.02047764 -0.0009969
  -0.02319363]
 [ 0.08221471 -0.00283498  0.05356892 ...  0.01659427 -0.01805034
   0.00998191]
 [-0.0590867   0.06246374 -0.00268008 ...  0.0429162  -0.08262999
   0.05348357]
 ...
 [-0.02785677  0.00917158 -0.06078977 ...  0.07465101  0.04480376
  -0.02368775]
 [ 0.01590867 -0.0231321  -0.00887359 ...  0.03156753 -0.08479695
  -0.04093414]
 [ 0.04486552  0.0894717  -0.0014197  ... -0.04229872 -0.02035187
  -0.01192394]]
b0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

In [125]:
with np.printoptions(threshold=np.inf):
    print(Y_test)

[45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 38 38 38 38 38 38 38 38 38 38 38 38
 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38

In [126]:
Y_test_predictions = predict(X_test, Y_test, *parameters)

Z0 = W0 @ X + b0 => (512, 1024) @ (1024, 13800) + (512, 1)
Z1 = W1 @ O0 + b1 => (256, 512) @ (512, 13800) + (256, 1)
Z2 = W2 @ O1 + b2 => (128, 256) @ (256, 13800) + (128, 1)
Predictions: [45 45 42 ...  9  9  9]
Prediction Accuracy = 0.7401449275362318


In [127]:
with np.printoptions(threshold=np.inf):
    print(Y_test_predictions)

[45 45 42 42 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 34 45 45 45 45 45
 45 45 45 42 45 45 45 45 45 42 45 45 10 45 20 13 45 45 45 45 45 42 45 45
 34 45 45 45 45 45 42 45 45 24 45 45  5 42 39 22 34 45 45 42 45 24 45 45
 45 29 45 25 45 12 45 45 45 45 45 20 45 12 45 45 45 45 45 24 45 45 39 39
 45 45 39 45 45 45 45 45 39 45 24 12 25 45 34 34 31 29 39 45 45 32 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
 45 24 45 45 45 45 45 45 45 45 45 45 45 45 12 45 45 45 45 45 45 45 45 45
 45 24 45 24 45 15 42 45 45 45 45 45 45 45 45 45 45 45 27 45 13 42 42 13
 45 45 13 45 42 45 45 45 45  5 45 22 25 45 12 45 45 45 45 45 45 45 10 45
 45 45 45 45 45 42 45 45 45 45 45 45 45 39 45 45 45 45 45 45 45 45 45 45
 45  9 45 45 45 45 45 45 20 20 12 12 38 38 38 38 38 38 38  8 37 41 29 23
 38 38 23 38 38 38 38 38 41 38 38 38 38 38 38 38 38